In [1]:
import tkinter as tk
from tkinter import filedialog
import cv2
import math
import mediapipe as mp
from keras.models import load_model
import numpy as np
import threading

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

model = load_model("C:/Users/DELL/Downloads/HAR_Dataset-20230601T155736Z-001/HAR_Dataset/har_model.h5")

pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)


def detectPose(image, pose):
    # Create a copy of the input image.
    output_image = image.copy()

    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform the Pose Detection.
    results = pose.process(imageRGB)

    # Retrieve the height and width of the input image.
    height, width, _ = image.shape

    # Initialize a list to store the detected landmarks.
    landmarks = []

    # Check if any landmarks are detected.
    if results.pose_landmarks:

        # Draw Pose landmarks on the output image.
#         mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
#                                   connections=mp_pose.POSE_CONNECTIONS)

        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:
            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                              (landmark.z * width)))

    return output_image, landmarks





def calculateAngle(landmark1, landmark2, landmark3):
    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))

    # Check if the angle is less than zero.
    if angle < 0:
        # Add 360 to the found angle.
        angle += 360

    # Return the calculated angle.
    return angle





def classifyPose(landmarks, output_image):
    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 0, 255)

    # Calculate the required angles.
    # ----------------------------------------------------------------------------------------------------------------

    # Get the angle between the left shoulder, elbow and wrist points.
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

    # Get the angle between the right shoulder, elbow and wrist points.
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])

    # Get the angle between the left elbow, shoulder and hip points.
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # Get the angle between the right hip, shoulder and elbow points.
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Get the angle between the left hip, knee and ankle points.
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

    # Get the angle between the right shoulder, knee and hip points
    right_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])

    # Get the angle between the left shoulder, knee and hip points
    left_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])



    
    angles = [round(left_elbow_angle, 2) / 360, round(right_elbow_angle, 2) / 360 ,
              round(left_shoulder_angle, 2) / 360, round(right_shoulder_angle, 2) / 360, 
              round(left_hip_angle, 2) / 360, round(right_hip_angle, 2) / 360, round(left_knee_angle, 2) / 360, 
              round(right_knee_angle, 2) / 360]
    feature_vector = np.array(angles).reshape(-1, 8)
    prediction = model.predict(feature_vector)
    
    probable_act = np.argmax(prediction[0])
    
    if probable_act == 0:
           label = 'Sitting'
           
    elif probable_act == 1:
           label = 'Standing'
           
    elif probable_act == 2:
           label = 'Walking'
           
    elif probable_act == 3:
           label = 'Waving Hands'
           
    elif probable_act  == 4:
           label = 'Yoga'
           
    elif probable_act == 5:
           label = 'Squats'
             
    

    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)

    # Write the label on the output image.
    cv2.putText(output_image, label, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    
    # print the pose details
    # print(f"{round(left_elbow_angle, 2)},{round(right_elbow_angle, 2)},{round(left_shoulder_angle, 2)},{round(right_shoulder_angle, 2)},{round(left_hip_angle, 2)},{round(right_hip_angle, 2)},{round(left_knee_angle, 2)},{round(right_knee_angle, 2)},Squats")

    return output_image, f'{label}'

class VideoPlayer:
    def __init__(self, master):
        self.master = master
        self.video_path = None
        self.cap = None
        self.playing = False

        master.configure(background="skyblue")

        window_width = 900
        window_height = 600

        # get the screen width and height
        screen_width = master.winfo_screenwidth()
        screen_height = master.winfo_screenheight()

        # calculate the x and y coordinates of the top-left corner of the window
        x = (screen_width // 2) - (window_width // 2)
        y = (screen_height // 2) - (window_height // 2)

        root.geometry(f"{window_width}x{window_height}+{x}+{y}")

        # create UI elements
        self.label = tk.Label(master, text="No video selected", font=("Arial", 20), bg="skyblue")
        self.label.pack(pady=10)

        self.button_select = tk.Button(master, text="Select video", command=self.select_video, font=("Arial", 20),
                                       bg="yellow")
        self.button_select.pack()

        self.button_play = tk.Button(master, text="Play", state="disabled", command=self.play_video, font=("Arial", 20),
                                     bg="white")
        self.button_play.pack(pady=10)

        self.button_select = tk.Button(master, text="Use Camera", command=self.webcam, font=("Arial", 20),
                                       bg="yellow")
        self.button_select.pack()
        
        footer_frame = tk.Frame(self.master, bg="black")
        footer_frame.pack(side="bottom", fill="x")

        footer_label = tk.Label(footer_frame, text="Developed By : Shivam Nikam, Mohit Wadekar, Aniket Borse, Chinmay Mahajan", font=("Arial", 12), bg="black", fg="white")
        footer_label.pack(pady=5)

    def select_video(self):
        self.video_path = filedialog.askopenfilename()
        self.label.config(text=self.video_path)
        self.cap = cv2.VideoCapture(self.video_path)
        self.button_play.config(state="normal")

    def select_cam(self):
        self.cap = cv2.VideoCapture(0)

    def play_video(self):
        if not self.playing:
            self.playing = True
            while True:
                
                ok, frame = self.cap.read()

                if not ok:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_height, frame_width, _ = frame.shape

                frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
                frame, landmarks = detectPose(frame, pose_video)

                if landmarks:
                    
                    frame, _ = classifyPose(landmarks, frame)

                cv2.imshow('Pose Classification', frame)

                if cv2.waitKey(25) & 0xFF == ord('q'):
                    self.label.configure(text="No Video Selected")
                    break
                    
            cv2.destroyAllWindows()
            self.cap.release()
            self.playing = False
            self.button_play.config(text="Play")
        else:
            self.playing = False

    def webcam(self):
        self.select_cam()
        if not self.playing:
            self.playing = True
            while True:
                ok, frame = self.cap.read()
                

                if not ok:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_height, frame_width, _ = frame.shape

                frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
                frame, landmarks = detectPose(frame, pose_video)

                if landmarks:
                    frame, _ = classifyPose(landmarks, frame)

                cv2.imshow('Pose Classification', frame)

                if cv2.waitKey(25) & 0xFF == ord('q'):
                    self.label.configure(text="No Video Selected")
                    break
            
            cv2.destroyAllWindows()
            self.cap.release()
            
    
if __name__ == '__main__':
    root = tk.Tk()
    root.title("Human Activity Recognition App")
    player = VideoPlayer(root)
    root.mainloop()

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.